# Fraxlend Market Analysis
    
This notebook visualizes lending rates and APRs across different markets (frxUSD and sfrxUSD).
It shows how different parameters affect the lending and borrowing rates in these markets.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Create interactive sliders
utilization_slider = widgets.FloatSlider(
    value=0.85,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Utilization Rate:',
    continuous_update=False
)

borrow_rate_slider = widgets.FloatSlider(
    value=0.10,
    min=0.0,
    max=0.20,
    step=0.01,
    description='Borrow APR:',
    continuous_update=False
)

sfrxusd_rate_slider = widgets.FloatSlider(
    value=0.08,
    min=0.0,
    max=0.20,
    step=0.01,
    description='sfrxUSD Rate:',
    continuous_update=False
)

# Display sliders
display(widgets.VBox([utilization_slider, borrow_rate_slider, sfrxusd_rate_slider]))

## Data Generation Functions

In [2]:
def frxUSDRates(utilization_rate, borrowRate, sfrxusdInterestRate):
    return  {
        'lentAPR': borrowRate * utilization_rate,
        'unlentAPR': 0,
        'borrowAPR': borrowRate
    };

def sfrxUSDRates(utilization_rate, borrowRate, sfrxusdInterestRate):
    # lentAPR = ((1 + borrowRate) / (1 + sfrxusdInterestRate)) - 1
    return {
        'lentAPR': borrowRate * utilization_rate,
        'unlentAPR': sfrxusdInterestRate * (1 - utilization_rate),
        'borrowAPR': borrowRate
    }


def getRates(utilization_rate, borrowRate, sfrxusdInterestRate):
    return {
        'frxUSDRates': frxUSDRates(utilization_rate, borrowRate, sfrxusdInterestRate),
        'sfrxUSDRates': sfrxUSDRates(utilization_rate, borrowRate, sfrxusdInterestRate)
    }

def calcfrxUSDBorrowRate(utilization_rate, lendRate, sfrxusdInterestRate):
    return  {
        'lentAPR': lendRate,
        'unlentAPR': 0,
        'borrowAPR': lendRate / (utilization_rate)
    };

def calcsfrxUSDBorrowRate(utilization_rate, lendRate, sfrxusdInterestRate):
    borrowRate = (lendRate - (sfrxusdInterestRate * (1 - utilization_rate))) / utilization_rate
    return {
        'lentAPR': borrowRate * utilization_rate,
        'unlentAPR': sfrxusdInterestRate * (1 - utilization_rate),
        'borrowAPR': borrowRate
    }

def getBorrowRates(utilization_rate, lendRate, sfrxusdInterestRate):
    return {
        'frxUSDRates': calcfrxUSDBorrowRate(utilization_rate, lendRate, sfrxusdInterestRate),
        'sfrxUSDRates': calcsfrxUSDBorrowRate(utilization_rate, lendRate, sfrxusdInterestRate)
    }

def generate_apr_comparison_data(current_interest_rate=0.05, sfrxusd_interest_rate=0.04):
    """
    Generate APR data for frxUSD and sfrxUSD markets across different utilization rates.
    
    Args:
        current_interest_rate (float): The current interest rate
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
    
    Returns:
        tuple: (DataFrame containing APR data, DataFrame containing borrow rates)
    """
    # Generate utilization rates from 0 to 1
    utilization_rates = np.linspace(0, 1, 21)  # 5% increments
    
    data = []
    borrow_rates = []
    for util in utilization_rates:
        rates = getRates(util, current_interest_rate, sfrxusd_interest_rate)
        
        # Add frxUSD data
        data.append({
            'utilization_rate': util,
            'market': 'frxUSD',
            'apr_type': 'lentAPR',
            'value': rates['frxUSDRates']['lentAPR']
        })
        data.append({
            'utilization_rate': util,
            'market': 'frxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['frxUSDRates']['unlentAPR']
        })
        
        # Add sfrxUSD data
        data.append({
            'utilization_rate': util,
            'market': 'sfrxUSD',
            'apr_type': 'lentAPR',
            'value': rates['sfrxUSDRates']['lentAPR']
        })
        data.append({
            'utilization_rate': util,
            'market': 'sfrxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['sfrxUSDRates']['unlentAPR']
        })
        
        # Add borrow rates
        borrow_rates.append({
            'utilization_rate': util,
            'market': 'frxUSD',
            'value': rates['frxUSDRates']['borrowAPR']
        })
        borrow_rates.append({
            'utilization_rate': util,
            'market': 'sfrxUSD',
            'value': rates['sfrxUSDRates']['borrowAPR']
        })
    
    return pd.DataFrame(data), pd.DataFrame(borrow_rates)

def generate_fixed_util_apr_data(utilization_rate=0.85, sfrxusd_interest_rate=0.04, max_borrow_rate=0.20):
    """
    Generate APR data for frxUSD and sfrxUSD markets across different borrow rates at fixed utilization.
    
    Args:
        utilization_rate (float): Fixed utilization rate (default 85%)
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
        max_borrow_rate (float): Maximum borrow rate to plot (default 20%)
    
    Returns:
        tuple: (DataFrame containing APR data, DataFrame containing borrow rates)
    """
    # Generate borrow rates from 0 to max_borrow_rate in 1% increments
    borrow_rates_array = np.linspace(0, max_borrow_rate, int(max_borrow_rate * 100) + 1)
    
    data = []
    borrow_rates_data = []
    
    for borrow_rate in borrow_rates_array:
        rates = getRates(utilization_rate, borrow_rate, sfrxusd_interest_rate)
        
        # Add frxUSD data
        data.append({
            'borrow_rate': borrow_rate,
            'market': 'frxUSD',
            'apr_type': 'lentAPR',
            'value': rates['frxUSDRates']['lentAPR']
        })
        data.append({
            'borrow_rate': borrow_rate,
            'market': 'frxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['frxUSDRates']['unlentAPR']
        })
        
        # Add sfrxUSD data
        data.append({
            'borrow_rate': borrow_rate,
            'market': 'sfrxUSD',
            'apr_type': 'lentAPR',
            'value': rates['sfrxUSDRates']['lentAPR']
        })
        data.append({
            'borrow_rate': borrow_rate,
            'market': 'sfrxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['sfrxUSDRates']['unlentAPR']
        })
        
        # Add borrow rates
        borrow_rates_data.append({
            'borrow_rate': borrow_rate,
            'market': 'frxUSD',
            'value': rates['frxUSDRates']['borrowAPR']
        })
        borrow_rates_data.append({
            'borrow_rate': borrow_rate,
            'market': 'sfrxUSD',
            'value': rates['sfrxUSDRates']['borrowAPR']
        })
    
    return pd.DataFrame(data), pd.DataFrame(borrow_rates_data)

def generate_lend_rate_comparison_data(utilization_rate=0.85, sfrxusd_interest_rate=0.08, max_lend_rate=0.20):
    """
    Generate APR data for frxUSD and sfrxUSD markets across different lend rates at fixed utilization.
    
    Args:
        utilization_rate (float): Fixed utilization rate (default 85%)
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
        max_lend_rate (float): Maximum lend rate to plot (default 20%)
    
    Returns:
        tuple: (DataFrame containing APR data, DataFrame containing borrow rates)
    """
    # Generate lend rates from 0 to max_lend_rate in 1% increments
    lend_rates_array = np.linspace(0, max_lend_rate, int(max_lend_rate * 100) + 1)
    
    data = []
    borrow_rates_data = []
    
    for lend_rate in lend_rates_array:
        rates = getBorrowRates(utilization_rate, lend_rate, sfrxusd_interest_rate)
        
        # Add frxUSD data
        data.append({
            'lend_rate': lend_rate,
            'market': 'frxUSD',
            'apr_type': 'lentAPR',
            'value': rates['frxUSDRates']['lentAPR']
        })
        data.append({
            'lend_rate': lend_rate,
            'market': 'frxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['frxUSDRates']['unlentAPR']
        })
        
        # Add sfrxUSD data
        data.append({
            'lend_rate': lend_rate,
            'market': 'sfrxUSD',
            'apr_type': 'lentAPR',
            'value': rates['sfrxUSDRates']['lentAPR']
        })
        data.append({
            'lend_rate': lend_rate,
            'market': 'sfrxUSD',
            'apr_type': 'unlentAPR',
            'value': rates['sfrxUSDRates']['unlentAPR']
        })
        
        # Add borrow rates
        borrow_rates_data.append({
            'lend_rate': lend_rate,
            'market': 'frxUSD',
            'value': rates['frxUSDRates']['borrowAPR']
        })
        borrow_rates_data.append({
            'lend_rate': lend_rate,
            'market': 'sfrxUSD',
            'value': rates['sfrxUSDRates']['borrowAPR']
        })
    
    return pd.DataFrame(data), pd.DataFrame(borrow_rates_data) 

## Visualization Functions

In [3]:
def plot_lending_rates(data, title="Lending Rates vs Utilization", save_path=None):
    """
    Create a line plot showing lending rates vs utilization rates for different markets.
    
    Args:
        data (pandas.DataFrame): DataFrame containing market data
        title (str): Title for the plot
        save_path (str, optional): Path to save the plot. If None, displays the plot.
    """
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    
    # Create the line plot
    sns.lineplot(
        data=data,
        x='utilization_rate',
        y='lending_rate',
        hue='market',
        linewidth=2.5
    )
    
    # Customize the plot
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel('Utilization Rate', fontsize=12)
    plt.ylabel('Lending Rate', fontsize=12)
    
    # Format axis labels as percentages
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
    
    # Add legend with custom styling
    plt.legend(title='Markets', title_fontsize=12, fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def plot_rate_comparison(data, utilization_points=[0.2, 0.5, 0.8, 0.95], save_path=None):
    """
    Create a bar plot comparing lending rates across markets at specific utilization points.
    
    Args:
        data (pandas.DataFrame): DataFrame containing market data
        utilization_points (list): List of utilization rates to compare
        save_path (str, optional): Path to save the plot. If None, displays the plot.
    """
    # Filter data for specific utilization points
    comparison_data = []
    for util in utilization_points:
        closest_idx = abs(data['utilization_rate'] - util).groupby(data['market']).idxmin()
        comparison_data.append(data.loc[closest_idx])
    
    comparison_df = pd.concat(comparison_data)
    
    plt.figure(figsize=(12, 6))
    sns.set_style("whitegrid")
    
    # Create the bar plot
    sns.barplot(
        data=comparison_df,
        x='market',
        y='lending_rate',
        hue='utilization_rate',
        palette='viridis'
    )
    
    # Customize the plot
    plt.title('Lending Rate Comparison Across Markets', fontsize=16, pad=20)
    plt.xlabel('Market', fontsize=12)
    plt.ylabel('Lending Rate', fontsize=12)
    
    # Format y-axis labels as percentages
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
    
    # Customize legend
    legend = plt.legend(title='Utilization Rate', title_fontsize=12, fontsize=10)
    for t in legend.get_texts():
        t.set_text('{:.0%}'.format(float(t.get_text())))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def plot_stacked_apr_comparison(data, borrow_rates, sfrxusd_interest_rate, title="APR Comparison: frxUSD vs sfrxUSD", save_path=None):
    """
    Create a stacked bar chart comparing total APRs (lentAPR + unlentAPR) for both markets,
    with borrow rate curves overlaid.
    
    Args:
        data (pandas.DataFrame): DataFrame containing APR data
        borrow_rates (pandas.DataFrame): DataFrame containing borrow rate data
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
        title (str): Title for the plot
        save_path (str, optional): Path to save the plot. If None, displays the plot.
    """
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Get unique utilization rates
    util_rates = sorted(data[data['market'] == 'frxUSD']['utilization_rate'].unique())
    
    # Set width of bars and positions of the bars
    width = 0.35
    x = np.arange(len(util_rates))
    
    # Prepare data for both markets
    markets = ['frxUSD', 'sfrxUSD']
    positions = [-width/2, width/2]  # Offset for side-by-side bars
    colors = {'frxUSD': ['#2ecc71', '#27ae60'], 'sfrxUSD': ['#3498db', '#2980b9']}
    
    for market, pos in zip(markets, positions):
        market_data = data[data['market'] == market]
        lent_data = market_data[market_data['apr_type'] == 'lentAPR']
        unlent_data = market_data[market_data['apr_type'] == 'unlentAPR']
        
        # Create bars - lentAPR at bottom, unlentAPR on top
        ax.bar(x + pos, lent_data['value'], width, 
               label=f'{market} Lent APR',
               color=colors[market][0])
        ax.bar(x + pos, unlent_data['value'], width, 
               bottom=lent_data['value'],
               label=f'{market} Unlent APR',
               color=colors[market][1],
               hatch='//' if market == 'sfrxUSD' else '')
    
    # Add single borrow rate line (using frxUSD market)
    market_borrow = borrow_rates[borrow_rates['market'] == 'frxUSD']
    ax.plot(x, market_borrow['value'], 
            label='Borrow APR',
            color='#e74c3c',
            linewidth=2.5,
            marker='o',
            markersize=4)
    
    # Add sfrxUSD interest rate line
    ax.axhline(y=sfrxusd_interest_rate, color='#8e44ad', linestyle='--', 
               label='sfrxUSD Interest Rate', linewidth=2)
    
    # Customize the plot
    ax.set_ylabel('APR', fontsize=12)
    ax.set_xlabel('Utilization Rate', fontsize=12)
    ax.set_title(title, fontsize=16, pad=20)
    
    # Set x-axis labels
    ax.set_xticks(x)
    ax.set_xticklabels([f'{rate:.0%}' for rate in util_rates])
    plt.setp(ax.get_xticklabels(), rotation=45)
    
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
    
    # Add legend
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Add grid for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    
    # Adjust layout
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def plot_fixed_util_apr_comparison(data, borrow_rates, sfrxusd_interest_rate, utilization_rate=0.85, title=None, save_path=None):
    """
    Create a stacked bar chart comparing total APRs (lentAPR + unlentAPR) for both markets
    across different borrow rates at fixed utilization.
    
    Args:
        data (pandas.DataFrame): DataFrame containing APR data
        borrow_rates (pandas.DataFrame): DataFrame containing borrow rate data
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
        utilization_rate (float): The fixed utilization rate used
        title (str): Title for the plot
        save_path (str, optional): Path to save the plot. If None, displays the plot.
    """
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Get unique borrow rates
    borrow_rates_list = sorted(data[data['market'] == 'frxUSD']['borrow_rate'].unique())
    
    # Set width of bars and positions of the bars
    width = 0.35
    x = np.arange(len(borrow_rates_list))
    
    # Prepare data for both markets
    markets = ['frxUSD', 'sfrxUSD']
    positions = [-width/2, width/2]  # Offset for side-by-side bars
    colors = {'frxUSD': ['#2ecc71', '#27ae60'], 'sfrxUSD': ['#3498db', '#2980b9']}
    
    for market, pos in zip(markets, positions):
        market_data = data[data['market'] == market]
        lent_data = market_data[market_data['apr_type'] == 'lentAPR']
        unlent_data = market_data[market_data['apr_type'] == 'unlentAPR']
        
        # Create bars - lentAPR at bottom, unlentAPR on top
        ax.bar(x + pos, lent_data['value'], width, 
               label=f'{market} Lent APR',
               color=colors[market][0])
        ax.bar(x + pos, unlent_data['value'], width, 
               bottom=lent_data['value'],
               label=f'{market} Unlent APR',
               color=colors[market][1],
               hatch='//' if market == 'sfrxUSD' else '')
    
    # Add single borrow rate line (using frxUSD market)
    market_borrow = borrow_rates[borrow_rates['market'] == 'frxUSD']
    ax.plot(x, market_borrow['value'], 
            label='Borrow APR',
            color='#e74c3c',
            linewidth=2.5,
            marker='o',
            markersize=4)
    
    # Add sfrxUSD interest rate line
    ax.axhline(y=sfrxusd_interest_rate, color='#8e44ad', linestyle='--', 
               label='sfrxUSD Interest Rate', linewidth=2)
    
    # Customize the plot
    ax.set_ylabel('APR', fontsize=12)
    ax.set_xlabel('Borrow Rate', fontsize=12)
    if title is None:
        title = f"APR Comparison at {utilization_rate:.0%} Utilization"
    ax.set_title(title, fontsize=16, pad=20)
    
    # Set x-axis labels
    ax.set_xticks(x[::5])  # Show every 5th label to avoid crowding
    ax.set_xticklabels([f'{rate:.0%}' for rate in borrow_rates_list[::5]])
    plt.setp(ax.get_xticklabels(), rotation=45)
    
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
    
    # Add legend
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Add grid for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    
    # Adjust layout
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def plot_lend_rate_apr_comparison(data, borrow_rates, sfrxusd_interest_rate, utilization_rate=0.85, title=None, save_path=None):
    """
    Create a stacked bar chart comparing total APRs (lentAPR + unlentAPR) for both markets
    across different lend rates at fixed utilization.
    
    Args:
        data (pandas.DataFrame): DataFrame containing APR data
        borrow_rates (pandas.DataFrame): DataFrame containing borrow rate data
        sfrxusd_interest_rate (float): The sfrxUSD interest rate
        utilization_rate (float): The fixed utilization rate used
        title (str): Title for the plot
        save_path (str, optional): Path to save the plot. If None, displays the plot.
    """
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Get unique lend rates
    lend_rates_list = sorted(data[data['market'] == 'frxUSD']['lend_rate'].unique())
    
    # Set width of bars and positions of the bars
    width = 0.35
    x = np.arange(len(lend_rates_list))
    
    # Prepare data for both markets
    markets = ['frxUSD', 'sfrxUSD']
    positions = [-width/2, width/2]  # Offset for side-by-side bars
    colors = {'frxUSD': ['#2ecc71', '#27ae60'], 'sfrxUSD': ['#3498db', '#2980b9']}
    line_colors = {'frxUSD': '#e74c3c', 'sfrxUSD': '#9b59b6'}
    
    for market, pos in zip(markets, positions):
        market_data = data[data['market'] == market]
        lent_data = market_data[market_data['apr_type'] == 'lentAPR']
        unlent_data = market_data[market_data['apr_type'] == 'unlentAPR']
        
        # Create bars - lentAPR at bottom, unlentAPR on top
        ax.bar(x + pos, lent_data['value'], width, 
               label=f'{market} Lent APR',
               color=colors[market][0])
        ax.bar(x + pos, unlent_data['value'], width, 
               bottom=lent_data['value'],
               label=f'{market} Unlent APR',
               color=colors[market][1],
               hatch='//' if market == 'sfrxUSD' else '')
        
        # Add borrow rate line for each market
        market_borrow = borrow_rates[borrow_rates['market'] == market]
        ax.plot(x, market_borrow['value'], 
                label=f'{market} Borrow APR',
                color=line_colors[market],
                linewidth=2.5,
                marker='o',
                markersize=4,
                linestyle='-' if market == 'frxUSD' else '--')
    
    # Add sfrxUSD interest rate line
    ax.axhline(y=sfrxusd_interest_rate, color='#8e44ad', linestyle='--', 
               label='sfrxUSD Interest Rate', linewidth=2)
    
    # Customize the plot
    ax.set_ylabel('APR', fontsize=12)
    ax.set_xlabel('Lend Rate', fontsize=12)
    if title is None:
        title = f"APR Comparison at {utilization_rate:.0%} Utilization"
    ax.set_title(title, fontsize=16, pad=20)
    
    # Set x-axis labels
    ax.set_xticks(x[::5])  # Show every 5th label to avoid crowding
    ax.set_xticklabels([f'{rate:.0%}' for rate in lend_rates_list[::5]])
    plt.setp(ax.get_xticklabels(), rotation=45)
    
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
    
    # Add legend
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Add grid for better readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    
    # Adjust layout
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show() 

In [4]:
# Create output widgets for each plot
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

def update_first_plot(change=None):
    with out1:
        clear_output(wait=True)
        plt.figure(figsize=(12, 8))
        
        # Generate visualization with current slider values
        apr_data, borrow_rates = generate_apr_comparison_data(
            borrow_rate_slider.value,
            sfrxusd_rate_slider.value
        )
        
        plot_stacked_apr_comparison(
            apr_data,
            borrow_rates,
            sfrxusd_rate_slider.value,
            title=f"APR Comparison: frxUSD vs sfrxUSD ({borrow_rate_slider.value:.0%} Borrow Rate)"
        )

# Connect sliders to update function
borrow_rate_slider.observe(update_first_plot, names='value')
sfrxusd_rate_slider.observe(update_first_plot, names='value')

# Display first output widget
display(widgets.HTML("<h2>APR Comparison: frxUSD vs sfrxUSD</h2>"))
display(out1)

# Initial plot
update_first_plot()

HTML(value='<h2>APR Comparison: frxUSD vs sfrxUSD</h2>')

Output()

In [5]:
def update_second_plot(change=None):
    with out2:
        clear_output(wait=True)
        plt.figure(figsize=(12, 8))
        
        # Generate visualization with current slider values
        fixed_util_data, fixed_util_borrow_rates = generate_fixed_util_apr_data(
            utilization_rate=utilization_slider.value,
            sfrxusd_interest_rate=sfrxusd_rate_slider.value
        )
        
        plot_fixed_util_apr_comparison(
            fixed_util_data,
            fixed_util_borrow_rates,
            sfrxusd_rate_slider.value,
            utilization_rate=utilization_slider.value,
            title=f"APR Comparison at {utilization_slider.value:.0%} Utilization"
        )

# Connect sliders to update function
utilization_slider.observe(update_second_plot, names='value')
sfrxusd_rate_slider.observe(update_second_plot, names='value')

# Display second output widget
display(widgets.HTML("<h2>APR Comparison by Utilization</h2>"))
display(out2)

# Initial plot
update_second_plot()

HTML(value='<h2>APR Comparison by Utilization</h2>')

Output()

In [6]:
def update_third_plot(change=None):
    with out3:
        clear_output(wait=True)
        plt.figure(figsize=(12, 8))
        
        # Generate visualization with current slider values
        lend_rate_data, lend_rate_borrow_rates = generate_lend_rate_comparison_data(
            utilization_rate=utilization_slider.value,
            sfrxusd_interest_rate=sfrxusd_rate_slider.value
        )
        
        plot_lend_rate_apr_comparison(
            lend_rate_data,
            lend_rate_borrow_rates,
            sfrxusd_rate_slider.value,
            utilization_rate=utilization_slider.value,
            title=f"APR Comparison by Lend Rate at {utilization_slider.value:.0%} Utilization"
        )

# Connect sliders to update function
utilization_slider.observe(update_third_plot, names='value')
sfrxusd_rate_slider.observe(update_third_plot, names='value')

# Display third output widget
display(widgets.HTML("<h2>APR Comparison by Lend Rate</h2>"))
display(out3)

# Initial plot
update_third_plot()

HTML(value='<h2>APR Comparison by Lend Rate</h2>')

Output()